# **Extração de Dados**

In [ ]:
import pandas as pd

# Dicionário com os caminhos para os arquivos.
arquivos = {
    '2021': 'data/recife-dados-receitas-2021.csv',
    '2022': 'data/recife-dados-receitas-2022.csv',
    '2023': 'data/recife-dados-receitas-2023.csv'
}

lista_dataframes = [] # Uma lista vazia para guardar os dados de cada ano temporariamente.

for ano, arquivo in arquivos.items():
    try:
        df_temp = pd.read_csv(
            arquivo,
            sep=';',           # Separador: CSVs do governo brasileiro usam ponto e vírgula.
            encoding='utf-8',  # Codificação: Para ler acentos (ç, ã, é) corretamente.
            thousands='.',     # Milhar: Avisa que o ponto serve para milhar (1.000).
            decimal=','        # Decimal: Avisa que a vírgula é decimal (0,50).
        )
       
        # Cria uma coluna nova para saber de qual ano veio aquele dado.
        df_temp['ano'] = ano

        # Adicionando esse pedaço na nossa lista.
        lista_dataframes.append(df_temp)

    except Exception as e:
        # Se der erro (arquivo não existe, nome errado), recebemos uma mensagem de erro.
        print(f"Erro em {ano}: {e}")

## **Consolidação/Concatenação**

In [ ]:
# Só rodamos se a lista não estiver vazia.
if lista_dataframes:
    # pd.concat = Concatenar.
    df_raw = pd.concat(lista_dataframes, ignore_index=True)
    print(f"Total de linhas no df unificado: {len(df_raw)}")

## **Lógica de Limpeza**

In [ ]:
def limpar_moeda(valor):
    if isinstance(valor, str): # Só limpa se for texto
        valor_limpo = valor.replace('R$', '')       # Tira o símbolo de Real.
        valor_limpo = valor_limpo.replace('.', '')  # Tira o ponto de milhar.
        valor_limpo = valor_limpo.replace(',', '')  # Troca vírgula por ponto, pois python só entende ponto decimal.

        return float(valor_limpo) # Conversão para número.
    return valor